In [2]:
# demo.ipynb

# Install the package (only needed if running locally, skip if installed globally)
%pip install -e .

import json
import pybamm
from BatteryModelMapper import OntologyParser, JSONLoader, JSONWriter, ParameterMapper

# Define URLs and file paths
ontology_url = 'https://w3id.org/emmo/domain/battery-model-lithium-ion/latest'
input_json_url = 'https://raw.githubusercontent.com/cidetec-energy-storage/cideMOD/main/data/data_Chen_2020/params_tuned_vOCPexpression.json'
output_json_path = 'converted_battery_parameters.json'
defaults_json_path = 'defaults_used.json'
template_url = 'https://raw.githubusercontent.com/BIG-MAP/ModelMapper/main/assets/bpx_template.json'
input_type = 'cidemod'
output_type = 'bpx'

# Initialize the OntologyParser
ontology_parser = OntologyParser(ontology_url)
mappings = ontology_parser.get_mappings(input_type, output_type)
#print("Mappings:", json.dumps({str(k): str(v) for k, v in mappings.items()}, indent=4))

# Load the input JSON file
input_data = JSONLoader.load(input_json_url)
#print("Input Data:", json.dumps(input_data, indent=4))

# Load the template JSON file
template_data = JSONLoader.load(template_url)

# Map the parameters using the mappings from the ontology
parameter_mapper = ParameterMapper(mappings, template_data, input_json_url, output_type, input_type)
output_data = parameter_mapper.map_parameters(input_data)
defaults_used_data = list(parameter_mapper.defaults_used)
#print("Output Data:", json.dumps(output_data, indent=4))

# Write the output JSON file
JSONWriter.write(output_data, output_json_path)

# Write the defaults used JSON file
JSONWriter.write(defaults_used_data, defaults_json_path)


## Test the parameters by running in PyBaMM
# Load the DFN model
model = pybamm.lithium_ion.DFN()

# Load the parameter values
parameter_values = pybamm.ParameterValues.create_from_bpx('converted_battery_parameters.json')

# Define the experiment: Charge from SOC=0.01, then discharge
experiment = pybamm.Experiment([
    ("Charge at C/5 until 4.2 V",
        "Hold at 4.2 V until 1 mA",
        "Rest for 1 hour",
        "Discharge at C/5 until 2.5 V")
])

# Create the simulation with the experiment
sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)

# Define initial concentration in negative and positive electrodes
parameter_values["Initial concentration in negative electrode [mol.m-3]"] = 0.0279 * parameter_values["Maximum concentration in negative electrode [mol.m-3]"]
parameter_values["Initial concentration in positive electrode [mol.m-3]"] = 0.9084 * parameter_values["Maximum concentration in positive electrode [mol.m-3]"]

# Solve the simulation
sim.solve()

# Plot the results
sim.plot()



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Obtaining file:///C:/Users/simonc/Documents/Github-local/FAIRData/ModelMapper
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: BatteryModelMapper
    Found existing installation: BatteryModelMapper 0.1
    Uninstalling BatteryModelMapper-0.1:
      Successfully uninstalled BatteryModelMapper-0.1
  Running setup.py develop for BatteryModelMapper
Note: you may need to restart the kernel to use updated packages.
Mapping added: ('positive_electrode', 'thickness', 'value') -> ('Parameterisation', 'Positive electrode', 'Thickness [m]')
Mapping added: ('positive_electrode', 'active_materials', 0, 'maximum_concentration', 'value') -> ('Parameterisation', 'Positive electrode', 'Maximum concentration [mol.m-3]')
Mapping added: ('positive_electrode', 'active_materials', 0, 'particle_radius', 'value') -> ('Parameterisation', 'Positive electrode', 'Particle radius [m]')
Mapping added: ('properties', 'external_surface_area'

interactive(children=(FloatSlider(value=0.0, description='t', max=13.194542491275975, step=0.13194542491275973…